<a href="https://colab.research.google.com/github/Mubassir1820/Kaggle_competition/blob/main/Kaggle_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,UID,col_0,col_1,col_2,col_3,col_4,col_5,col_6,y
0,0,A 0,B0,C2,D1,100,E1,F2,237000
1,1,A1,B0,C11,D4,100,E4,F2,86193
2,2,A0,B0,C18,D0,0,E0,F2,169200
3,3,A2,B0,C11,D1,100,E1,F2,58000
4,4,A0,B0,C67,D1,0,E1,F2,235000


In [4]:
df.shape

(2628, 9)

In [5]:
df['col_0'].value_counts()

A0        1517
A1         489
A2         178
A3          67
 A0         53
A 0         38
AO          36
A0          35
A 1         21
A1          16
 A1         16
 A2          7
A 2          6
A2           4
 A3          4
A 3          1
A   3        1
Name: col_0, dtype: int64

In [6]:
df['col_1'].value_counts()

B0        2375
 B0         48
B 0         33
B0          33
BO          29
B3          10
B2           7
B1           5
B   3        1
B1           1
Name: col_1, dtype: int64

In [7]:
df['col_2'].value_counts()

C11    728
C2     587
C4     416
C9     204
C15     77
      ... 
C62      1
C63      1
C92      1
C76      1
C66      1
Name: col_2, Length: 90, dtype: int64

In [8]:
df['col_3'].value_counts()

D1     2093
D4      116
D0       61
D2       60
D6       54
       ... 
D32       1
D71       1
D75       1
D66       1
D73       1
Name: col_3, Length: 68, dtype: int64

In [9]:
df['col_5'].value_counts()

E1     2120
E4      121
E2       62
E0       57
E6       44
       ... 
E45       1
E47       1
E20       1
E67       1
E62       1
Name: col_5, Length: 67, dtype: int64

In [10]:
df['col_6'].value_counts()

F2        1976
F0         286
F1          97
F 2         46
F2          43
 F2         39
F0           7
F 0          6
 F0          6
FO           4
F 1          3
F1           2
 F1          1
Name: col_6, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2628 entries, 0 to 2627
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UID     2628 non-null   int64 
 1   col_0   2489 non-null   object
 2   col_1   2542 non-null   object
 3   col_2   2628 non-null   object
 4   col_3   2628 non-null   object
 5   col_4   2628 non-null   int64 
 6   col_5   2628 non-null   object
 7   col_6   2516 non-null   object
 8   y       2628 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 184.9+ KB


In [12]:
df.describe()

,UID,col_4,y
count,2628.000000,2628.000000,2628.000000
mean,1313.500000,46.308980,137642.657154
std,758.782578,48.558188,62886.926575
min,0.000000,0.000000,5132.000000
25%,656.750000,0.000000,95000.000000
50%,1313.500000,0.000000,135000.000000
75%,1970.250000,100.000000,176325.000000
max,2627.000000,100.000000,423000.000000


In [13]:
df.value_counts()

UID   col_0   col_1  col_2  col_3  col_4  col_5  col_6  y     
0     A 0     B0     C2     D1     100    E1     F2     237000    1
1752  A0      B0     C18    D2     0      E2     F2     248100    1
1745  A0      B0     C2     D1     100    E1     F2     185000    1
1746  A1      B0     C11    D1     100    E1     F2     60400     1
1747   A0     B0     C20    D1     100    E1     F0     100000    1
                                                                 ..
864   A0      B 0    C39    D1     100    E1     F1     190000    1
863   A0      B0     C11    D1     100    E1     F2     220110    1
862   A0      B0     C11    D1     100    E1     F2     160000    1
861   A0      B0     C11    D1     0      E1     F2     63000     1
2627  A0      B0     C2     D1     100    E1     F2     191475    1
Length: 2301, dtype: int64

In [14]:
df.isnull().sum()


UID        0
col_0    139
col_1     86
col_2      0
col_3      0
col_4      0
col_5      0
col_6    112
y          0
dtype: int64

In [15]:
# Replace the missing values with mode
df['col_0'].fillna(df['col_0'].mode()[0], inplace=True)
df['col_1'].fillna(df['col_1'].mode()[0], inplace=True)
df['col_6'].fillna(df['col_6'].mode()[0], inplace=True)

In [16]:
df.isnull().sum()

UID      0
col_0    0
col_1    0
col_2    0
col_3    0
col_4    0
col_5    0
col_6    0
y        0
dtype: int64

In [17]:
# Regular expression to clean the data
import re

def preprocess(text):
  text = re.sub('[a-z^\s]','',text)
  return text

In [18]:
df['col_0'] = df['col_0'].map(preprocess)

In [19]:
df['col_0'].value_counts()

A0    1782
A1     542
A2     195
A3      73
AO      36
Name: col_0, dtype: int64